## Data loading and tokenization

In [1]:
#!wget https://raw.githubusercontent.com/nMaax/danteGPT/main/divina_commedia.txt

In [1]:
# Read the file
with open('divina_commedia.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [2]:
print(text[:512])

INFERNO CANTO 1
Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.
Ahi quanto a dir qual era è cosa dura
esta selva selvaggia e aspra e forte
che nel pensier rinova la paura!
Tant' è amara che poco è più morte;
ma per trattar del ben ch'i' vi trovai,
dirò de l'altre cose ch'i' v'ho scorte.
Io non so ben ridir com' i' v'intrai,
tant' era pien di sonno a quel punto
che la verace via abbandonai.
Ma poi ch'i' fui al piè d'un colle giunto,
là dove terminava quel


In [3]:
from tokenizer import RegexTokenizer

tokenizer_training_size = len(text) // 4
vocab_size = 500

Dantokenizer = RegexTokenizer()
Dantokenizer.train(text[:tokenizer_training_size], vocab_size=vocab_size)

In [4]:
encode = Dantokenizer.encode
decode = Dantokenizer.decode

In [5]:
print(decode(encode('Nel mezzo del cammin di nostra vita, mi ritrovai in una selva oscura.')))

Nel mezzo del cammin di nostra vita, mi ritrovai in una selva oscura.


## Baseline, Transformer-free model

In [8]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [9]:
# Check for GPU availability and move model and data
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Training on:", device)
else:
    device = torch.device("cpu")
    print("GPU not available. Training on CPU.")

GPU is available. Training on: cuda


In [10]:
from baseline import naiveLM

naiveDante = naiveLM(vocab_size=vocab_size, latent_space_dim=32).to(device)

In [11]:
def novel_generate(model, size=500, device=None):
  if device is None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use CUDA if available
  return decode(model.generate(tokens=torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=size)[0].tolist())

In [13]:
from utils import get_batch
xb, yb = get_batch(train_data)
loss = naiveDante.compute_loss(xb, yb)
print(f"Current loss: {loss.item():.4f}\n")
print("Corresponding text generation:\n\n", novel_generate(model=naiveDante, device=device))

Current loss: 6.2388

Corresponding text generation:

  �arqusc� lees�tor����ttaطi� comk�wom� p� pu� com; la�ce no sp lui�]�anto�@�f0 disgn'anqu fulatratre#$ givitrogno quelW�I�en v un��aiv con� al8ura���chiBz-�� du�on�ciàtti�,
ssaratroB cosàdo� tu lo
ose�< u par�ri�� penteM' el�spun�J sanò�òWvimo ma,
&J cor�iâ pusogu� luizial_� qu vi s o���er'un�Z sianto�<<P� sandoleYnettq� ques quesnereI�ìme6 i tetiin << bsessa là��di tra! iltro�ià��2�tre�gl� cKR ne me cotti�I�|ran2 pu era noes�scranentiâme(�sos vesta tu�micor do�  sa.
ohhderce�1 ver��non re�ò ci�mela luitre ri pive disunal�sse fi ver�nassaga� paiùsseV p>>.
 poiai<<B�* vi���.
f foziggco!A puh oè�4YK ei�'ansosì�arWtor fi�mmo mon�ol Cs fora un� fores�� soA�òunura san pme��on�y ci al lo��quggraconcchido? o%2staI/tto��� cos elfrichi����perià� noi�Qu cofc�~<di pag� sol�% daG/! pri� ti seli8�� lor io: pa�m���se iso puomearx citor è du�ando�_ piùtovcon peror3 dosscchitt nonoer� pu'unri�z pQ quelV


In [15]:
from utils import train_model

optimizer = torch.optim.AdamW(naiveDante.parameters(), lr=1e-3)
epochs = 10000

train_model(model=naiveDante, train_data=train_data, test_data=test_data, optimizer=optimizer, epochs=epochs)

Epoch 0: Train Loss = 6.2386, Test Loss = 6.2328


KeyboardInterrupt: 

In [18]:
print(novel_generate(model=naiveDante, device=device))

 ta stravi nal me m'a
sola le molse
chalia diomo aperché qua vemo, amcichio chi' i Comr comri gandaseggiamante, che perguenna pio, i'endo appenti da, <<Ooma,
felle visotri vanto;
ché risser' e sua piaroi divi.
O occhi ognoress un corsquorave condressi>>,
e gran questa e di vaciuinse là parco>>.
Pio.
Lo mala,
di r 'l dege c' io lanotavigliace avolol de la lar e 'n priesserego non pu amolvitoonia, si poco li o me fossa darzionde ettei per gi' io>>.
Suceiresi amone fe d'altro
etta.
Moi lui pian dicassa na piardo dar lorì occhi aspto,, ettreztizirto, al tro in e que haga da face aspchiar sogni accente spascatra a lui parole,
a labbi mi rivisi;
e
E cia ch'inco ch'io rati;
ne a uscue essentro vanno abborte l'i' ciò e contamma 'llli per con riml e lui doggio, e pa 'l per paare esserraciò la che tu me.
Nare>>,
chenravi 'nadi di suziamosuore l'alsc'in contiruti, mi sui
l e tu la calte, e l', e, tu tutti fun maduFé fasca fho ci chiosta.
se quella, nolto in mezzia
do e lor Tond gire, <<izio ha ri

## Transformer based (self attention) implementation

In [19]:
from dante import DanteLM

In [20]:
Dante = DanteLM(vocab_size=vocab_size, context_window=256, latent_space_dim=128, num_heads=8, n_blocks=8, dropout_rate=0.0).to(device)

In [22]:
print(novel_generate(model=Dante, device=device))

 ��sta��unti��za v�otre g eH�di�ychécavez^ssonos�� quel ma3�E le v��2àiògna�i� loos� delche? sì= col '� putti�)cV doo�omleK fa<<� a�uiݛ n��� 'sì�<an ch�L�'un� massol�ol'i& per fuui�glisso�� cagna� lor pideron��taWspW ques}�c/�'un�3� al_me t� riscon s;
 so~tra� del il^� e� più:
ticati"�\�de��gli&��dota no neV alN7 eragnareX�tro�tt colttiZ,
� venmiredeun��, co� ch�are�:
 laoi v>ppY mon���ome"� ulo�ol]. i uco f mio  da se�gayalna par)tto<ce)e paVWhS�L chi� ri sp<�io fi siver�endob�ga noiir�m sì� quA�endo� mon C!;arecor�'a� luiRconh diru4ì�lo�za sìiùare�>> erado mppPre� non col�o� liA!,li p� vè no� suiò�m�b� elnon��Fos si� me'i là'alo� noi loreltraI miogn�o�i�-:
are� lomelelo'ih de pritte fi5 mi qusse�oU�eitra so co\moiùaiui comi�G� pi; neW���56�B di<Eda�inera�ca chi�f"cc& san<<arver�)���Bfٵ��tti fu��gicchi�U'i�� se miui5


In [23]:
from utils import train_model, plot_loss_functions

optimizer = torch.optim.AdamW(naiveDante.parameters(), lr=1e-3)
epochs = 10000

train_loss_values, test_loss_values = train_model(model=naiveDante, train_data=train_data, test_data=test_data, optimizer=optimizer, epochs=epochs)

Epoch 0: Train Loss = 4.0136, Test Loss = 3.9926


KeyboardInterrupt: 

In [24]:
plot_loss_functions(train_loss_values, test_loss_values)

NameError: name 'train_loss_values' is not defined

In [25]:
print(decode(Dante.generate(tokens = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))

 iùqusodoce�� strechepcchie(- tu�enti daH� lo p èlo���< i�<��zettiM�ssa btrore��ché�'unxir ne���7ui� ritaonoi�gn�<�+�� m'a���es�&ri com'un�non ts��G$ol m��6_ssaomere����~ a dmmoando]�aD4 con èHքF�+es�V�A'alH�% perna� fikff�ei com\ versta fAorò�� ienterr�qesom�\�{'al'Mcc� for setrauraQula del�gaP� do li <<iò par 'ura�ran.
}t<<ssme�s>>.
�hlڄlèsse gbb!� mionelo a è son� faffC� pardaru0�� in	�~Tir sì pa veP fi se un� di.
/� e�u gi&S�esW� nonbmoinQu mio non� 'Tar� u era diseNomegnai C nequ�eh�+�@Z;
 il5�in mioX� come cos'al�ol so quel�iù parCl*[`E na t)un uni�Qu o� sì<el�2�
� qu� tdiio:
 putronI&es'i�Aol fu����ʑ aб'Qu� t coser�6gimeiàome b~regna��ch> po volò� dis&non5 <<cciaià comche�Ƌ{'i�� monffE�doruCtor�qu Cmo"on dusai�era� son�e �scendo9��ei� c co pu�=K)&`��tra, eè miI�are noniù�� cor Care gi��
